# TP FINAL Debandi

## Carga de datos y preparación del dataset

En este trabajo utilizo los datos de Covid-19 junto con un dataset complementario de información sobre cuando cada persona es migrante o no.

Este notebook levanta tres archivos: 
casosCovid: es un archivo de casos de marzo 2020 hasta mayo 2021. Este archivo fue bajado desde el sitio de datos abiertos del Ministerio de Salud de Argentina
http://datos.salud.gob.ar/dataset?groups=covid-19

migrantes2020 y migrantes2021: ambos archivos corresponden a dos pedidos de acceso a la información publica realizados en 2021. Este archivo contempla el ID-caso y algunos campos para dar cuenta de la nacionalidad y el país de nacimiento de cada persona.

La cantidad de registros del archivo de casos es dee 15.071.716 
Dado el volumen y el tiempo que llevaba este procesamiento de manera local, para este trabajo se realizó una muesta del 10%.


Esta notebook levanta los archivos de casos y toma una muestra del 10%., Hace un join con los datos de migrantes y se seleccionan las variables de interés.
Luego se guardan los datos en formato parquet. En la notebook de análisis se toman directamente estos datos.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("pyspark-intro")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

# Levanto la base de casos

Levanto el dataset de casos con los tipos de cada columna para poder trabajarlos

In [3]:
#especifico el esquema para poder definir los tipos

##no se pudo utilizar este esquema, traia un error al utilizarlo en el que el parquet se guardaba vacío posteriormente.

from pyspark.sql.types import *

schema = (StructType()
          .add("id_evento_caso", IntegerType())
          .add("sexo", StringType())
          .add("edad", IntegerType())
          .add("edad_años_meses", IntegerType())
          .add("residencia_pais_nombre", StringType())
          .add("residencia_provincia_nombre", StringType())
          .add("residencia_departamento_nombre", StringType())
          .add("carga_provincia_nombre", StringType())
          .add("fecha_inicio_sintomas", DateType())

          .add("fecha_apertura", DateType())
          .add("sepi_apertura", IntegerType())
          .add("fecha_internacion", DateType())
          .add("cuidado_intensivo", StringType())
          .add("fecha_cui_intensivo", DateType())
          .add("fallecido", StringType())
          .add("fecha_fallecimiento", DateType())
          
          .add("asistencia_respiratoria_mecanica", DateType())
          .add("carga_provincia_id", IntegerType())
          .add("origen_financiamiento", StringType())
          .add("clasificacion", StringType())
          .add("clasificacion_resumen", StringType())
          .add("residencia_provincia_id", DateType())
          .add("fecha_diagnostico", DateType())
          .add("residencia_departamento_id", IntegerType())
          .add("ultima_actualizacion", IntegerType())

         )

In [25]:
#levanto los casos
dfcasosCompleto = spark.read.csv("/dataset/covid/Covid19Casos.csv", header=True,inferSchema = True)

#dfcasosCompleto = spark.read.csv("/dataset/covid/Covid19Casos.csv", 
#                        schema=schema, header=True)



In [28]:
dfcasosCompleto.printSchema()

root
 |-- id_evento_caso: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- edad_años_meses: string (nullable = true)
 |-- residencia_pais_nombre: string (nullable = true)
 |-- residencia_provincia_nombre: string (nullable = true)
 |-- residencia_departamento_nombre: string (nullable = true)
 |-- carga_provincia_nombre: string (nullable = true)
 |-- fecha_inicio_sintomas: timestamp (nullable = true)
 |-- fecha_apertura: timestamp (nullable = true)
 |-- sepi_apertura: integer (nullable = true)
 |-- fecha_internacion: timestamp (nullable = true)
 |-- cuidado_intensivo: string (nullable = true)
 |-- fecha_cui_intensivo: timestamp (nullable = true)
 |-- fallecido: string (nullable = true)
 |-- fecha_fallecimiento: timestamp (nullable = true)
 |-- asistencia_respiratoria_mecanica: string (nullable = true)
 |-- carga_provincia_id: integer (nullable = true)
 |-- origen_financiamiento: string (nullable = true)
 |-- clasificacion: string (nu

In [27]:

dfcasosCompleto.show(2)


+--------------+----+----+---------------+----------------------+---------------------------+------------------------------+----------------------+---------------------+-------------------+-------------+-----------------+-----------------+-------------------+---------+-------------------+--------------------------------+------------------+---------------------+---------------+---------------------+-----------------------+-------------------+--------------------------+--------------------+
|id_evento_caso|sexo|edad|edad_años_meses|residencia_pais_nombre|residencia_provincia_nombre|residencia_departamento_nombre|carga_provincia_nombre|fecha_inicio_sintomas|     fecha_apertura|sepi_apertura|fecha_internacion|cuidado_intensivo|fecha_cui_intensivo|fallecido|fecha_fallecimiento|asistencia_respiratoria_mecanica|carga_provincia_id|origen_financiamiento|  clasificacion|clasificacion_resumen|residencia_provincia_id|  fecha_diagnostico|residencia_departamento_id|ultima_actualizacion|
+-----------

In [29]:
dfcasosCompleto.count()

15071716

Hay un total de 15.071.716 casos.

Cambio los tipos de las columnas. NO pude importarlo con el schema. Me trajo multiples errores

In [30]:
#Randomly sample 10% of the data without replacement
dfcasos = dfcasosCompleto.sample(False, 0.10, seed=7)

In [31]:
dfcasos.count()

1507618

Cantidad de casos luego del sampling del 10%: 1.507.618

In [32]:
#dfTemp2.write.parquet("/dataset/parquet/covidcasos-confirmados.parquet") 

# levanto el dataset de migrantes

Voy a levantar ahora los dos archivos de migrantes, unirlos. Estos archivos corresponden a dos pedidos de información distintos.

In [33]:
schema_mig = (StructType()
          .add("id_caso", IntegerType())
          .add("nacionalidad", StringType())
          .add("pais_nacimiento", StringType())
          .add("tipo_doc", StringType())
          .add("nro_doc", IntegerType())
          .add("doc_argentino", IntegerType())
          .add("base", IntegerType())
         )

In [34]:
#dfmig1 = spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2020.csv", header=True)


In [35]:
dfmig1 = (spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2020.csv", 
                         schema=schema_mig, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True))


In [36]:
#dfmig1.show()

In [37]:
dfmig2 = (spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2021.csv", 
                         schema=schema_mig, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True))

In [38]:
dfmig2.printSchema()

root
 |-- id_caso: integer (nullable = true)
 |-- nacionalidad: string (nullable = true)
 |-- pais_nacimiento: string (nullable = true)
 |-- tipo_doc: string (nullable = true)
 |-- nro_doc: integer (nullable = true)
 |-- doc_argentino: integer (nullable = true)
 |-- base: integer (nullable = true)



In [39]:
dfmig= dfmig1.union(dfmig2)

# To remove the duplicates:

#result = result.dropDuplicates()


In [40]:
dfmig.printSchema()

root
 |-- id_caso: integer (nullable = true)
 |-- nacionalidad: string (nullable = true)
 |-- pais_nacimiento: string (nullable = true)
 |-- tipo_doc: string (nullable = true)
 |-- nro_doc: integer (nullable = true)
 |-- doc_argentino: integer (nullable = true)
 |-- base: integer (nullable = true)



In [41]:
dfmig.count()

11437754

La cantidad de registros de la union de los archivos de migrantes es de 11437754

## Creación del dataset final

Se hace un join entre ambos dataframes y se seleccionan las variables de interés

In [42]:
##Creo las dos vistas para poder hacer una consulta sql
dfmig.createOrReplaceTempView("dfmig")
dfcasos.createOrReplaceTempView("dfcasos")

##realizo el join entre ambos por id-caso. Hago left join para quedarme con todos los casos aun cuando no tienen dato de migrante
dfTempCasos= spark.sql("""
    SELECT id_evento_caso,sexo, edad,
    fecha_inicio_sintomas,fecha_apertura,fecha_diagnostico,
    sepi_apertura,origen_financiamiento,
    residencia_provincia_id,residencia_departamento_id,
    fecha_internacion,cuidado_intensivo,fallecido,
    pais_nacimiento
    FROM dfcasos LEFT JOIN dfmig on dfcasos.id_evento_caso=dfmig.id_caso
    WHERE clasificacion_resumen='Confirmado'
    """)


In [43]:
dfTempCasos.show()

+--------------+----+----+---------------------+-------------------+-------------------+-------------+---------------------+-----------------------+--------------------------+-------------------+-----------------+---------+---------------+
|id_evento_caso|sexo|edad|fecha_inicio_sintomas|     fecha_apertura|  fecha_diagnostico|sepi_apertura|origen_financiamiento|residencia_provincia_id|residencia_departamento_id|  fecha_internacion|cuidado_intensivo|fallecido|pais_nacimiento|
+--------------+----+----+---------------------+-------------------+-------------------+-------------+---------------------+-----------------------+--------------------------+-------------------+-----------------+---------+---------------+
|        740861|   F|  75|  2020-03-02 00:00:00|2020-03-07 00:00:00|2020-03-09 00:00:00|           10|              Público|                     22|                       140|               null|               NO|       NO|      Argentina|
|        744751|   M|  67|  2020-03-08 0

In [ ]:
#dfmig.count()
#vuelvo a chequear la distribución de fallecido
dfTempCasos.groupBy('fallecido').count().orderBy('count').show()

In [45]:
dfTempCasos.printSchema()

root
 |-- id_evento_caso: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha_inicio_sintomas: timestamp (nullable = true)
 |-- fecha_apertura: timestamp (nullable = true)
 |-- fecha_diagnostico: timestamp (nullable = true)
 |-- sepi_apertura: integer (nullable = true)
 |-- origen_financiamiento: string (nullable = true)
 |-- residencia_provincia_id: integer (nullable = true)
 |-- residencia_departamento_id: integer (nullable = true)
 |-- fecha_internacion: timestamp (nullable = true)
 |-- cuidado_intensivo: string (nullable = true)
 |-- fallecido: string (nullable = true)
 |-- pais_nacimiento: string (nullable = true)



In [46]:
dfTempCasos.count()

474572

Al hacer un LEFT JOIN  y filtrar por confirmados nos quedamos con 474.572 casos

## Creación archivo parquet

In [48]:
dfTempCasos.write.parquet("/dataset/parquetv4/covidmigrantes.parquet") 